In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

# Monthly Data

In [8]:
#Read Data
Exportaciones_netas = pd.read_csv("../csv_results/Exportaciones_Mensuales_Netas.csv")
Incidentes_geopoliticos = pd.read_csv('../csv_results/incidentes_geopoliticos.csv')
Terrorismo = pd.read_csv("../csv_results/Terrorism.csv")

Variaciones_crudo = pd.read_csv("../csv_results/Variaciones_Crudo_Mensual")
Variaciones_NGPL = pd.read_csv('../csv_results/Variaciones_NGPL_Mensual')
Variaciones_Others = pd.read_csv('../csv_results/Variaciones_Others_Mensual')
Variaciones_Refinado = pd.read_csv('../csv_results/Variaciones_Refinery_Mensual')

#Delete unnecesary columns in the variaciones Data set
del Variaciones_crudo['Unnamed: 0']
del Variaciones_NGPL['Unnamed: 0']
del Variaciones_Others['Unnamed: 0']
del Variaciones_Refinado['Unnamed: 0']

#Delete unnecesary columns in the Incidentes geopoliticos DB
del Incidentes_geopoliticos["Unnamed: 0"]

#Delete unnecesary columns in the Expotaciones_netas DB
Exportaciones_netas = Exportaciones_netas.drop(columns=["Unnamed: 0", "Exp_notes", 'Imp_notes'])

#Delete unnecesary columns in the Terrorismo DB
Terrorismo = Terrorismo.drop(columns=['Unnamed: 0', 'imonth', 'approxdate', 'targtype1', 'targsubtype1', 'doubtterr', 'alternative', 'propextent'])

In [10]:
#Undo pivot tables
Variaciones_crudo = Variaciones_crudo.set_index("Country").stack().reset_index()
Variaciones_NGPL = Variaciones_NGPL.set_index("Country").stack().reset_index()
Variaciones_Others = Variaciones_Others.set_index("Country").stack().reset_index()
Variaciones_Refinado = Variaciones_Refinado.set_index("Country").stack().reset_index()

In [11]:
###Create a list with definitive countries names 
#Problematic contries names are marked with a variant in parenthesis
countries = ['Albania','Algeria','Angola','Argentina','Azerbaijan','Benin','Bolivia','Brazil','Brunei','Cameroon','Canada','China','Colombia','Republic of the Congo', 'Democratic Republic of the Congo', 'Denmark','Ecuador','Egypt','Estonia','Gabon','Georgia','Ghana','Guatemala','Indonesia','Iran','Iraq','Kazakhstan','Kuwait','Kyrgyzstan','Libya','Malaysia','Mexico','Mongolia', 'Myanmar (Burma)','Nigeria','Norway','Oman','Qatar','Russia (Soviet Union)','Saudi Arabia','South Sudan','Sudan','Trinidad and Tobago','Tunisia','Turkmenistan','United Arab Emirates','United Kingdom','Venezuela','Vietnam','Yemen (North Yemen)']

#### Create a dictionary of countries names who may be replaced
#First create a dictionary with countries in each db
db_countries = { "var" : Variaciones_crudo["Country"].unique().tolist(),
                "exp" : Exportaciones_netas['Country'].unique().tolist(),
                "inc" : Incidentes_geopoliticos['country'].unique().tolist(),
                "ter" : Terrorismo['country_txt'].unique().tolist() }

#Then for each db_list check which countries do not match with our list and put them in a dictionary
to_change = {'var' : [x for x in db_countries["var"] if x not in countries], 
                    'exp' : [x for x in db_countries["exp"] if x not in countries],
                    'inc' : [x for x in db_countries["inc"] if x not in countries],
                    'ter' : [x for x in db_countries["ter"] if x not in countries] }

#Print the dictionary in order to find which names need to be replaced
to_change

{'var': ['Burma', 'Congo-Brazzaville', 'Congo-Kinshasa', 'Russia', 'Yemen'],
 'exp': ['Brunei Darussalam',
  'Bolivia, Plurinational State of',
  'Iran, Islamic Republic of',
  'Myanmar',
  'Russian Federation',
  'Venezuela, Bolivarian Republic of',
  'Viet Nam',
  'Yemen'],
 'inc': ['Congo', 'DR Congo (Zaire)'],
 'ter': ['Myanmar', 'Yemen', 'Russia']}

In [12]:
#Now replace the names in each db

#Exportaciones netas
Exportaciones_netas["Country"] = Exportaciones_netas["Country"].replace({'Brunei Darussalam' : 'Brunei',
  'Bolivia, Plurinational State of' : 'Bolivia',
  'Iran, Islamic Republic of' : 'Iran' ,
  'Myanmar' : 'Myanmar (Burma)',
  'Russian Federation' : 'Russia (Soviet Union)',
  'Venezuela, Bolivarian Republic of' : 'Venezuela',
  'Viet Nam' : 'Vietnam',
  'Yemen' : 'Yemen (North Yemen)'}) 

#Incidentes geopoliticos
Incidentes_geopoliticos["country"] = Incidentes_geopoliticos["country"].replace({'Congo' : 'Republic of the Congo', 'DR Congo (Zaire)' : 'Democratic Republic of the Congo'})

#Terrorismo
Terrorismo["country_txt"] = Terrorismo["country_txt"].replace({'Myanmar' : 'Myanmar (Burma)', 'Yemen' : 'Yemen (North Yemen)' , 'Russia' : 'Russia (Soviet Union)', 'South Vietnam' : 'Vietnam' })

#Variaciones en el valor 
Variaciones_crudo["Country"] = Variaciones_crudo["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_NGPL["Country"] = Variaciones_NGPL["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_Others["Country"] = Variaciones_Others["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_Refinado["Country"] = Variaciones_Refinado["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})


In [57]:
#Analisemos las distintas bases de datos

periods = [Exportaciones_netas['Month'], Incidentes_geopoliticos['Start_month'], Terrorismo['period'], Variaciones_crudo['level_1'], Variaciones_NGPL['level_1'],Variaciones_Others['level_1'], Variaciones_Refinado['level_1']]


analisis = pd.DataFrame({'nombres' : ['Exportaciones_netas', 'Incidentes_geopoliticos', 'Terrorismo', 'Variaciones_crudo', 'Variaciones_NGPL', 'Variaciones_Others', 'Variaciones_Refinado'],
                                'extension' : [len(x) for x in periods],
                                'maxima' : [np.max(x) for x in periods],
                                'minima' : [np.min(x) for x in periods] }, 
                                columns=["nombres",'extension','maxima', 'minima'])
analisis.sort_values("minima")

,nombres,extension,maxima,minima
2,Terrorismo,1605,2018-12,1972-01
1,Incidentes_geopoliticos,52989,2019-12,1989-01
0,Exportaciones_netas,9102,2020-06,2002-01
3,Variaciones_crudo,21200,Sep 2019,Apr 1973
4,Variaciones_NGPL,16400,Sep 2019,Apr 1993
5,Variaciones_Others,16400,Sep 2019,Apr 1993
6,Variaciones_Refinado,16400,Sep 2019,Apr 1993


# Next spets:
### DataFrames Globales
- Una tabla de terrorismo vs valores de crudo desde abril del 73 hasta diciembre de 2018
- Una tabla de terrorismo e incidentes vs todas las variaciones de valores dede abril del 92 hasta diciembre de 2018
- Una tabla de exportaciones netas vs valores, incidentes y terroismo desde enero 2002 hasta diciembre de 2018
- Un tabla de exportaciones netas vs valores e incidentes desde enero 2002 hasta septiembre de 2019

### DataFrames por paises
- Pendiente de como se va hacer



# Yearly Data